In [4]:
import cv2
import numpy as np
import tensorflow as tf
import winsound # 소리 내는 라이브러리
import pickle # 외부 파일을 생성하고 저장, 불러오는 라이브러리
import sys # 외부 파일 제어 라이브러리
from PyQt5.QtWidgets import * # 윈도우 프로그램 GUI 라이브러리

## 훈련된 모델 가져오기
cnn_model = tf.keras.models.load_model('cnn_for_stanford_dogs.h5')
## 견종 이름 데이터 가져오기(wb는 수정 가능, rb는 읽기만 가능)
dog_species = pickle.load(open('dog_species_names.txt', 'rb'))

## PyQt를 사용한 메인 App 클래스 정의
class DogSpeciesRecognition(QMainWindow):
    # 생성자
    def __init__(self):
        super().__init__() # QMainWindow(부모) 클래스 상속
        self.setWindowTitle('DOG_SPECIES') # 제목
        self.setGeometry(200,200,700,100) # 크기

        ## 버튼 추가
        fileButton = QPushButton('Photo Open', self)
        recognitionButton = QPushButton('Recognition', self)
        quitButton = QPushButton('Quit', self)

        ## 버튼 위치
        fileButton.setGeometry(10,10,100,30) # 앞에 두개는 위치값 / 뒤에 두개는 크기값
        recognitionButton.setGeometry(110,10,100,30)
        quitButton.setGeometry(510,10,100,30)

        ## 버튼 기능 부여
        fileButton.clicked.connect(self.picktureOpenFunction)
        recognitionButton.clicked.connect(self.recognitionFunction)
        quitButton.clicked.connect(self.quitFunction)

        ## Method 정의
        def picktureOpenFunction(self):
            fname = QFileDialog.getOpenFileName(self, 'Picture read', './')
            self.img = cv2.imread(fname[0])
            if self.img in None:
                sys.exit('Not Found File')
            cv2.imshow('Dog Image', self.img)

        def recognitionFunction(self):
            X = np.reshape(cv2.resize(self.img, (224, 224)), (1, 224, 224, 3))
            res = cnn_model.predict(X)[0] # 모델의 예측
            top5 = np.argsort(-res)[:5] # 오름차순 정렬(descending)
            top5_dog_species_names = [dog_species[i] for i in top5]
            for i in range(5):
                prob = f'({str(res[top5[i]])})' ## 5개의 확률
                name = str(top5_dog_species_names[i]).split('-')[1] ## 견종 이름
                
                ## 예측 결과 이미지에 텍스트 추가
                cv2.putText(self.img, prob+name, (10, 100, + i*30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.imshow('Dog Image', self.img) # 결과 이미지 추가
            winsound.Beep(1000, 500) # 소리 추가
        
        def quitFunction(self):
            cv2.destroyAllWindows() # cv2창 끄는거
            self.close() # 윈도우창 끄는거

app = QApplication(sys.argv)
win = DogSpeciesRecognition()
win.show()
app.exec_()

TypeError: Error when deserializing class 'Conv2D' using config={'name': 'conv1/conv', 'trainable': True, 'dtype': 'float32', 'filters': 64, 'kernel_size': [7, 7], 'strides': [2, 2], 'padding': 'valid', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'groups': 1, 'activation': 'linear', 'use_bias': False, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}.

Exception encountered: Argument `name` must be a string and cannot contain character `/`. Received: name=conv1/conv (of type <class 'str'>)